# From tabular data to binned data

## Loading tabular data

In this tutorial we will use a file created by a McStas simulation for a diamond sample, with Geant4 simulating the detectors.

We can use [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) to load the table as a [pandas.Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).
It is much faster than [numpy.loadtxt](https://numpy.org/doc/stable/reference/generated/numpy.loadtxt.html) and also slightly more convenient:

In [ ]:
import pandas as pd

filename = 'https://public.esss.dk/groups/scipp/scippneutron/4/data_dream_diamond.zip'
df = pd.read_csv(filename, sep='\t')
df

`scipp.compat.from_pandas` can convert the `pandas.Dataframe` to a `scipp.Dataset`.
The column names encode the physical units so we must extract them manually:

In [ ]:
import scipp as sc

ds = sc.compat.from_pandas(df)
for key in list(ds.keys()):
    name, *remainder = key.split(' ')
    ds[name] = ds.pop(key)
    ds[name].unit = remainder[0][1:-1] if remainder else None
ds.coords.pop('row')  # we have no use for this row index
sc.table(ds[:10])

This 1-D dataset represents the tabular data that was read from the file.
In the above table, each row (record) describes and *event*, in this case the detection of a neutron, with its associated metadata such as the detector module or the x, y, and z position.

To be able to histogram or bin data by a column, scipp must know which columns are metadata and which column holds data values.
The table is thus actually a table of metadata values for events with an implicit data value of "1 count" each.
To continue we convert this into a data array:

In [ ]:
da = sc.DataArray(sc.ones(sizes=ds.sizes, unit='counts'))
for name in ds:
    da.coords[name] = ds[name].data
da

## Histogramming and binning

We are now ready to bin or histogram our data.
Scipp uses the following terminology:

- Binning preserves the original data records in a list per bin.
- Histogramming adds up the values from all contributing records into a single value per bin.

In both cases we need to define bin edges.
As an initial 1-D example, we will compute a wavelength histogram:

In [ ]:
wavelength = sc.linspace(
    'wavelength',
    da.coords['wavelength'].min().value,
    da.coords['wavelength'].max().value,
    num=1001,
    unit='angstrom',
)

This can be histogrammed using `sc.histogram`:

In [ ]:
histogrammed = sc.histogram(da, bins=wavelength)
histogrammed

Alternatively, we can use `sc.bin`, which keeps the underlying events and their metadata:

In [ ]:
binned = sc.bin(da, edges=[wavelength])
binned

Since we used the same bin edges for histogramming and binning, computing the sum of values within each bin (given by `binned.bins.sum()`) gives the same result as histogramming directly.
Therefore only a single line is visible in the following plot:

In [ ]:
sc.plot({'histogrammed': histogrammed, 'binned': binned.bins.sum()})

Since the above combines data from all pixels, the wavelength distribution is not really meaningful.

While the result of histogramming may appear similar or identical, the internal structure is very different.
The histogrammed data consists of essentially two arrays, one for the values (yellow) and one for the wavelengths (green):

In [ ]:
sc.show(histogrammed)

The top level structure of the binned data is the same, i.e., we have and array of values and an array of wavelengths.
The difference is that each value (bin) stores all contributing table rows:

In [ ]:
sc.show(binned)

### Exercise 1

- Define bin edges for `z_pos` and histogram and bin `da`.
  Plot the results.
- Define different bin edges for `z_pos`, e.g., with more values.
  Use `sc.bin` with the new edges *on the result of the previous binning*, i.e., *not* on the original table `da`.
  Why is this possible?

In [ ]:
# your code here

## Multi-dimensional binning

### Spatial binning

`scipp.bin` can handle multiple dimensions:

In [ ]:
x = da.coords['x_pos']
y = da.coords['y_pos']
z = da.coords['z_pos']
x_pos = sc.linspace('x_pos', x.min().value, x.max().value, num=31, unit='mm')
y_pos = sc.linspace('y_pos', y.min().value, y.max().value, num=31, unit='mm')
z_pos = sc.linspace('z_pos', z.min().value, z.max().value, num=31, unit='mm')
binned = sc.bin(da, edges=[z_pos, y_pos, x_pos])
binned['z_pos', 20:].bins.sum().plot(norm='log', aspect='equal')

Above we can see a cut through the detector, which has the shape of a thick cylinder mantle.
The advantage of binned data over histogrammed data is that the meta data for each underlying event is still present.
We can therefore change the binning, or bin in additional dimensions.
For example, we can select the slice containing $z = 0$ and turn it into a higher-resolution cut:

In [ ]:
x_fine = sc.linspace('x_pos', x.min().value, x.max().value, num=41, unit='mm')
y_fine = sc.linspace('y_pos', y.min().value, y.max().value, num=101, unit='mm')
z_slice = binned['z_pos', sc.scalar(0.0, unit='mm')]
xy_cut = sc.bin(z_slice, edges=[y_fine, x_fine])
xy_cut

In [ ]:
xy_cut.bins.sum().transpose().plot(aspect='equal')

#### Exercise 2

TODO

In [ ]:
# your code here

### Logical binning

Above we binned according to x, y, and z.
This reflects neither the physics nor the logical structure of the detectors and is generally not very useful.
The original table additionally contains information about the logical structure of the detector array.
In this case it is divided into modules, segments, counters, wires, and strips.
We define:

In [ ]:
groups = {
    dim: sc.arange(dim, 1, da.coords[dim].max().value + 1, unit=None, dtype='int64')
    for dim in ['module', 'segment', 'counter', 'wire', 'strip']
}
groups

Instead of using `scipp.bin` with the `edges` keyword argument we can use the`groups` keyword argument to perform a binning based on discrete values.
The result is 5-D:

In [ ]:
binned = sc.bin(da, groups=list(groups.values()))
binned

#### Exercise 3

- Group only by strip and wire.
- Plot the result.
- Wires is radial, strip is along z. TODO
  Explain the plot, e.g., why is intensity decreasing with increasing wire index?

In [ ]:
# your code here

In [ ]:
sc.bin(da, groups=[groups['strip'], groups['wire']]).bins.sum().plot()

Our data contains, for each detected neutron, the position of the associated voxel.
It is more practical to store this for every bin (voxel) instead of very every event.
We can also combine the x, y, and z components into a single array of position vectors:

In [ ]:
pos = sc.zeros(sizes=binned.sizes, dtype=sc.DType.vector3, unit='mm')
pos.fields.x = binned.bins.coords['voxel_x'].bins.mean()
pos.fields.y = binned.bins.coords['voxel_y'].bins.mean()
pos.fields.z = binned.bins.coords['voxel_z'].bins.mean()
binned.coords['position'] = pos
binned

This can be used to create a 3-D scatter plot:

In [ ]:
binned.bins.sum().plot(projection='3d', positions='position', pixel_size=10)

We can also inspect an individual component such as a strip:

In [ ]:
binned['strip', 200].bins.sum().plot(projection='3d', positions='position', pixel_size=10)

#### Exercise 4

Make scatter plots of various bits

In [ ]:
# your code here

### Binning based on bin edges and groups combined

It is also possible to "group" and "bin" at the same time.
Since strips roughly  create a correspond to scattering angle, a plot against wavelength and strip may be useful:

In [ ]:
sc.bin(da, groups=[groups['strip']], edges=[wavelength]).bins.sum().plot(norm='log')